In [1]:
from pyEmceeFit import *

In [2]:
sampler,pos,prob,state,flat_samples = main(True)

running with 4 threads


  0%|          | 0/100 [00:00<?, ?it/s]/home/kirk/anaconda3/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]
 16%|█▌        | 16/100 [00:00<00:00, 151.69it/s]

running burn-in


  2%|▏         | 16/1000 [00:00<00:06, 150.79it/s]

production run (n = 1000 iterations)


100%|██████████| 1000/1000 [00:07<00:00, 142.75it/s]


In [3]:
import matplotlib.pyplot as plt
import numpy as np

In [4]:
 for θ in flat_samples[np.random.randint(len(flat_samples), size=10)]:
        print("i = {0:.2f}, rMin = {1:.2f}, MFac = {2:.2f}, rFac = {3:.2f}, f1 (sin^2) = {4:.2f}, f2 (cos^2) = {5:.2f}, f3 (sin*cos)= {6:.2f}".format(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6]))

i = 64.14, rMin = 1033.98, MFac = 1.19, rFac = 2.01, f1 (sin^2) = -0.37, f2 (cos^2) = 0.55, f3 (sin*cos)= 0.19
i = 59.85, rMin = 996.08, MFac = 0.84, rFac = 2.13, f1 (sin^2) = 0.12, f2 (cos^2) = 0.18, f3 (sin*cos)= 0.20
i = 83.23, rMin = 1115.71, MFac = 0.94, rFac = 0.33, f1 (sin^2) = -0.41, f2 (cos^2) = 0.19, f3 (sin*cos)= -0.37
i = 54.11, rMin = 844.56, MFac = 1.01, rFac = 1.40, f1 (sin^2) = 1.36, f2 (cos^2) = 0.32, f3 (sin*cos)= 0.26
i = -13.86, rMin = 1065.72, MFac = 0.99, rFac = 1.65, f1 (sin^2) = 1.43, f2 (cos^2) = 1.29, f3 (sin*cos)= -0.23
i = 59.85, rMin = 996.08, MFac = 0.84, rFac = 2.13, f1 (sin^2) = 0.12, f2 (cos^2) = 0.18, f3 (sin*cos)= 0.20
i = 22.30, rMin = 1140.98, MFac = 1.10, rFac = 1.01, f1 (sin^2) = 0.03, f2 (cos^2) = 0.56, f3 (sin*cos)= 0.22
i = 94.76, rMin = 966.04, MFac = 0.43, rFac = 2.46, f1 (sin^2) = 0.20, f2 (cos^2) = 0.08, f3 (sin*cos)= 0.28
i = 79.33, rMin = 882.83, MFac = 0.62, rFac = 0.37, f1 (sin^2) = -0.05, f2 (cos^2) = 0.60, f3 (sin*cos)= 0.54
i = -10.4

In [11]:
def plotParams(data,θList):
    λCen=2.172; ν = (data[0]-λCen)/λCen*3e5
    indx=[0,1,2,6,7,8,12,13,14,18,19,20]; oindx=[3,4,5,9,10,11,15,16,17,21,22,23]
    fig,(ax1,ax2,ax3) = plt.subplots(nrows=3,ncols=1,figsize=(12,7))
    ax1.errorbar(ν,data[3],yerr=data[6],marker="o",ms=3,label="3C 273")
    ax2.errorbar(ν,np.mean(np.array(data[4])[indx],axis=0),yerr=ν*0+0.07,label="3C 273",marker="o",ms=3)
    ax3.errorbar(ν,np.mean(np.array(data[4])[oindx],axis=0),yerr=ν*0+0.07,label="3C 273",marker="o",ms=3)
    for θ in θList:µ
        i,rMin,Mfac,rFac,f1,f2,f3 = θ
        line,phase,phaseo = getProfiles(ν,θ,data)
        ax1.plot(ν,line,label="disk wind",lw=2)
        ax2.plot(ν,phase,label="disk wind",lw=2)
        ax3.plot(ν,phaseo,label="disk wind",lw=2)
        
    return fig,ax1,ax2,ax3

In [ ]:
θList = flat_samples[np.random.randint(len(flat_samples),size=10)]
data = readPickle("3c273_juljanmarmay_append_gilles_specirf_wide_v6.p")
fig,ax1,ax2,ax3 = plotParams(data,θList)